In [ ]:
import pandas as pd
import numpy as np
import re
from google.cloud import bigquery

import warnings
warnings.filterwarnings ('ignore')

### Отправка в BQ Raw Data (generated)

In [ ]:
raw = pd.read_excel('dataset.xlsx', usecols = ['location', 'sales', 'date'])

In [ ]:
raw['months_cyrilc'] = [re.findall('[\u0401\u0451\u0410-\u044f]+', i) for i in raw['date']]

for i in range(len(raw)):
    raw['months_cyrilc'][i] = raw['months_cyrilc'][i][0]

mapping = dict({"января": 1,
           "февраля": 2,
           "марта": 3,
           "апреля": 4,
           "мая": 5,
           "июня": 6,
           "июля": 7,
           "августа": 8,
           "сентября": 9,
           "октября": 10,
           "ноября": 11,
           "декабря": 12})

raw['months_num'] = raw['months_cyrilc'].map(mapping)

raw['date_adj'] = 0
for i in range(len(raw)):
    raw['date_adj'][i] = raw['date'][i].split()[2]+'/'+raw['months_num'][i].astype(str)+'/'+raw['date'][i].split()[0]

raw['date_adj'] = pd.to_datetime(raw['date_adj'], format= "mixed")
raw = raw.drop({'date', 'months_cyrilc', 'months_num'}, axis = 1)

raw['sales'] = [re.sub('\s+', '', i) for i in raw['sales']]
raw['sales'] = raw['sales'].astype('int32')


In [ ]:
#Dataset with Table name variable
BqDatasetwithtable='Seasonality.Raw_data'
#BQproject name variable
BqProject='spartan-thunder-311714'
# pandas-gbq method to load data
# append data if data table exists in BQ project
# set chunk size of records to be inserted
raw.to_gbq(BqDatasetwithtable, BqProject, chunksize=20000, if_exists= 'replace') #'append' )

### Получение из BQ Raw Data для обработки и насыщения

In [ ]:
PROJECT_ID = 'spartan-thunder-311714' # Change to your project.
REGION = 'US'

In [ ]:
%%bigquery df --project $PROJECT_ID

SELECT
    *
FROM
  `spartan-thunder-311714.Seasonality.Raw_data`

In [ ]:
Q1 = df.groupby(['location'])['sales'].quantile(0.25)
Q3 = df.groupby(['location'])['sales'].quantile(0.75)
IQR = Q3-Q1

df['month'] = df['date_adj'].dt.month
for l in IQR.index.tolist():
  # Calculate the lower and upper bounds for the current location 'l'
  lower_bound = Q1[l] - 1.5 * IQR[l]
  upper_bound = Q3[l] + 1.5 * IQR[l]

  df.loc[df['location'] == l, 'Sales adjusting 1 step'] = np.where((df.loc[df['location'] == l, 'sales'] < lower_bound) | \
                              (df.loc[df['location'] == l, 'sales'] > upper_bound),
                              df.loc[df['location'] == l].groupby(['month'])['sales'].transform(lambda x: x.median()),
                              df.loc[df['location'] == l, 'sales'])

df['year'] = df['date_adj'].dt.year
df['variance'] = df.groupby(['location', 'year'])['Sales adjusting 1 step'].transform(lambda x: x.std() / x.mean())

df['_avg_within_month_indx'] = df.groupby(['month', 'location'])['Sales adjusting 1 step'].transform('mean')
df['Sales adjusting step 2'] = np.where((df['variance']>0.25) & (abs(df['Sales adjusting 1 step']/df['_avg_within_month_indx']-1)>=0.25),
                                          df['_avg_within_month_indx'],
                                          df['Sales adjusting 1 step'])


df= df.sort_values(by = ['location', 'date_adj'], ascending = True)
min_year = df['year'].min()
df['Sales CumSum 12M'] = df.groupby(['location'])['Sales adjusting step 2'].transform(lambda x: x.rolling(window = 12).sum())
df['Sales CumSum 12M'] = np.where(df['year']==min_year, df['Sales CumSum 12M'].fillna(df[df['year']==min_year].groupby(['location'])['Sales CumSum 12M'].transform('max')), df['Sales CumSum 12M'])

df['Sales Moving Average'] = round(df['Sales CumSum 12M'] /12,0)
df['Sales Moving Avg Smoothed'] = round(df.groupby(['location'])['Sales Moving Average'].transform(lambda x: x.rolling(2, min_periods = 1).mean()),0)
df['Seasonal component'] = np.where(df['Sales Moving Avg Smoothed'].notna(), df['Sales adjusting step 2']-df['Sales Moving Avg Smoothed'], np.nan)
df['Avg Seasonal component'] = np.where(df['Sales Moving Avg Smoothed'].notna(), df.groupby(['location', 'month'])['Seasonal component'].transform('mean'), np.nan)

avg = pd.pivot_table(df,
                     #columns = ['month'],
                     index = ['location','month'],
                     values = ['Avg Seasonal component'],
                     aggfunc = 'mean')
avg = avg.reset_index().groupby(['location'])['Avg Seasonal component'].sum()/12
avg = avg.reset_index().rename(columns = {"Avg Seasonal component": "corr_coeff"})

df= pd.merge(df, avg, on = "location", how = "left")
df['Avg Seasonal Component adjusted'] = df['Avg Seasonal component'] - df['corr_coeff']

df['Sales MovAvg Smoothed 12M'] = round(df.groupby(['location', 'month'])['Sales Moving Avg Smoothed'].transform('mean'),0)
df['Sales MovAvg Smoothed 12M w SeasComp'] = df['Sales MovAvg Smoothed 12M'] + df['Avg Seasonal Component adjusted']
df['max seas comp'] = df.groupby(['location'])['Avg Seasonal Component adjusted'].transform('max')
df['Season months'] =np.where(df['Avg Seasonal Component adjusted']>=df['max seas comp']*0.5, 1, np.nan)
#т.к. привели сезон.компон. и сглаженн.скользящую ср.продаж к средним по индексу месяца, то по умолчанию рассматривается 1 год. Поэтому нужно явно указать год, например, 2023.
# максимальная сезон.комп - как кульминац.сезона

In [ ]:
new_cols = {}
for col in df.columns:
    new_name = re.sub('\s+', '_',col)
    #new_cols.append(col+":"+new_name)
    new_cols.update({
        col: new_name
    })
df.rename(columns = new_cols, inplace=True)

### Отправка в BQ очищенных и обогащенных данных

In [ ]:
#Dataset with Table name variable
BqDatasetwithtable='Seasonality.Enriched_raw_data'
#BQproject name variable
BqProject='spartan-thunder-311714'
# pandas-gbq method to load data
# append data if data table exists in BQ project
# set chunk size of records to be inserted
df.to_gbq(BqDatasetwithtable, BqProject, chunksize=20000, if_exists= 'replace') #'append' )

### создание summary в BQ через Python + var : создание summary в Superset

In [ ]:
PROJECT_ID = 'spartan-thunder-311714' # Change to your project.
REGION = 'US'

In [ ]:

%%bigquery  --project $PROJECT_ID

CREATE OR REPLACE VIEW Seasonality.Summary_python AS (
select
location,
max(case when `Season_months` = 1 then `Avg_Seasonal_Component_adjusted` else null end) as max_season_comp,
sum(case when `Season_months` = 1 then `Season_months` else null end) as season_months,
min(case when `Season_months` = 1 then `month` else null end) as season_start_month,
avg(case when `Season_months` = 1 then `Avg_Seasonal_Component_adjusted` else null end) as average_season_comp,
NTILE(5) over (ORDER BY avg(case when `Season_months` = 1 then `Avg_Seasonal_Component_adjusted` else null end) desc) as `Group`,
(avg(case when `Season_months` = 1 then `Sales_MovAvg_Smoothed_12M_w_SeasComp` else null end)/avg(case when `Season_months` is null then `Sales_MovAvg_Smoothed_12M_w_SeasComp` else null end))-1 as `Season_vs_NotSeason`
from `spartan-thunder-311714.Seasonality.Enriched_raw_data`
where  `year` = 2023
group by 1);